In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import numpy as np
import pandas as pd

In [3]:
## Data Wrangling
porosity_df = pd.read_csv('Testing/X-poro15_55_5.csv',header=None)
porosity_np = porosity_df.to_numpy()
porosity_np = porosity_np[:,0:4125]
porosity_list = []


In [4]:
for i in range(250):
    new_data_infil = np.reshape(porosity_np[i,:],(15,55,5))
    porosity_list.append(new_data_infil)
    

In [5]:
saturation_df = pd.read_csv('Testing/X-saturation15_55_5.csv',header=None)
saturation_np = saturation_df.to_numpy()
saturation_np = saturation_np[:,0:4125]
saturation_list = []


In [6]:
for i in range(250):
     new_data_infil = np.reshape(saturation_np[i,:],(15,55,5))
     saturation_list.append(new_data_infil)
    

In [7]:
saturation_df.shape

(250, 4126)

In [8]:
pressure_df = pd.read_csv('Testing/X-pressure15_55_5.csv',header=None)
pressure_np = pressure_df.to_numpy()
pressure_np = pressure_np[:,0:4125]
pressure_list = []


In [9]:
for i in range(250):
    new_data_infil = np.reshape(pressure_np[i,:],(15,55,5))
    pressure_list.append(new_data_infil)
    

In [10]:
permeability_df = pd.read_csv('Testing/X-perm.csv',header=None)
permeability_np = permeability_df.to_numpy()
permeability_np = permeability_np[:,0:4125]
permeability_list = []


In [11]:
for i in range(250):
    new_data_infil = np.reshape(permeability_np[i,:],(15,55,5))
    permeability_list.append(new_data_infil)
    


In [12]:
auxiliary_df = pd.read_csv('Testing/X-auxilary15_55_5_test.csv',header=None)
auxiliary_np = auxiliary_df.to_numpy()
auxiliary_np = auxiliary_np[:,0:7]
auxiliary_list = []

In [14]:
for i in range(250):
    auxiliary_list.append(auxiliary_np[i,:])
    

In [15]:
FOPT_df = pd.read_csv('Testing/Y-FOPT.csv',header=None)
FOPT_np = FOPT_df.to_numpy()
FOPT_np = FOPT_np[:,0]
FOPT_list = []


FileNotFoundError: [Errno 2] File b'Testing/Y-FOPT.csv' does not exist: b'Testing/Y-FOPT.csv'

In [15]:
for i in range(250):
    FOPT_list.append([FOPT_np[i]])
    

In [16]:
### (1): Data
training_X_perm = np.array(permeability_list)
training_X_poro = np.array(porosity_list)
training_X_satu = np.array(saturation_list)
training_X_pres = np.array(pressure_list)
training_X_auxu = np.array(auxiliary_list)
training_Y = np.array(FOPT_list)/10000.0



validation_X_perm = np.array(permeability_validation_list)
validation_X_poro = np.array(porosity_validation_list)
validation_X_satu = np.array(saturation_validation_list)
validation_X_pres = np.array(pressure_validation_list)
validation_X_auxu = np.array(auxiliary_validation_list)
validation_Y = np.array(FOPT_validation_list)/10000.0

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
### Format Channels for CNN Networks
training_X_perm = np.expand_dims(training_X_perm, axis=1)
training_X_poro = np.expand_dims(training_X_poro, axis=1)
training_X_satu = np.expand_dims(training_X_satu, axis=1)
training_X_pres = np.expand_dims(training_X_pres, axis=1)
pt_training_X_perm = Variable(torch.from_numpy(training_X_perm).float(), requires_grad=False).to(device)
pt_training_X_poro = Variable(torch.from_numpy(training_X_poro).float(), requires_grad=False).to(device)
pt_training_X_satu = Variable(torch.from_numpy(training_X_satu).float(), requires_grad=False).to(device)
pt_training_X_pres = Variable(torch.from_numpy(training_X_pres).float(), requires_grad=False).to(device)
pt_training_X_auxu = Variable(torch.from_numpy(training_X_auxu).float(), requires_grad=False).to(device)
pt_training_Y = Variable(torch.from_numpy(training_Y).float(), requires_grad=False).to(device)




validation_X_perm = np.expand_dims(validation_X_perm, axis=1)
validation_X_poro = np.expand_dims(validation_X_poro, axis=1)
validation_X_satu = np.expand_dims(validation_X_satu, axis=1)
validation_X_pres = np.expand_dims(validation_X_pres, axis=1)
validation_Y = validation_Y.reshape(-1,1)
pt_validation_X_perm = Variable(torch.from_numpy(validation_X_perm).float(), requires_grad=False).to(device)
pt_validation_X_poro = Variable(torch.from_numpy(validation_X_poro).float(), requires_grad=False).to(device)
pt_validation_X_satu = Variable(torch.from_numpy(validation_X_satu).float(), requires_grad=False).to(device)
pt_validation_X_pres = Variable(torch.from_numpy(validation_X_pres).float(), requires_grad=False).to(device)
pt_validation_X_auxu = Variable(torch.from_numpy(validation_X_auxu).float(), requires_grad=False).to(device)
pt_validation_Y = Variable(torch.from_numpy(validation_Y).float(), requires_grad=False).to(device)

In [18]:
class CNN100(nn.Module):
    def __init__(self):
        super(CNN100, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels=1, out_channels=100, kernel_size=[3,3,2], stride=1, padding=0)
        self.conv2 = torch.nn.Conv3d(in_channels=100, out_channels=100, kernel_size=[3,3,2], stride=1, padding=0)
        self.conv3 = torch.nn.Conv3d(in_channels=100, out_channels=100, kernel_size=[3,3,2], stride=1, padding=0)
        self.batchnorm = torch.nn.BatchNorm3d(num_features=1)
        self.maxpool = torch.nn.MaxPool3d(kernel_size=[2,2,1])
        self.maxpool_2 = torch.nn.MaxPool3d(kernel_size=[1,2,2])
        

    def forward(self, x):
        norm_x = self.batchnorm(x)
        conv1_out = self.maxpool(torch.nn.functional.relu(self.conv1(norm_x)))
        conv2_out = self.maxpool(torch.nn.functional.relu(self.conv2(conv1_out)))
        conv3_out = self.maxpool_2(torch.nn.functional.relu(self.conv3(conv2_out)))
        flat_out = conv3_out.reshape(conv3_out.shape[0],-1)
        return flat_out

class CNN20(nn.Module):
    def __init__(self):
        super(CNN20, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels=1, out_channels=20, kernel_size=[3,3,2], stride=1, padding=0)
        self.conv2 = torch.nn.Conv3d(in_channels=20, out_channels=20, kernel_size=[3,3,2], stride=1, padding=0)
        self.conv3 = torch.nn.Conv3d(in_channels=20, out_channels=20, kernel_size=[3,3,2], stride=1, padding=0)
        self.batchnorm = torch.nn.BatchNorm3d(num_features=1)
        self.maxpool = torch.nn.MaxPool3d(kernel_size=[2,2,1])
        self.maxpool_2 = torch.nn.MaxPool3d(kernel_size=[1,2,2])

    def forward(self, x):
        norm_x = self.batchnorm(x)
        conv1_out = self.maxpool(torch.nn.functional.relu(self.conv1(norm_x)))
        conv2_out = self.maxpool(torch.nn.functional.relu(self.conv2(conv1_out)))
        conv3_out = self.maxpool_2(torch.nn.functional.relu(self.conv3(conv2_out)))
        flat_out = conv3_out.reshape(conv3_out.shape[0],-1)
        return flat_out

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.perm_cnn = CNN100()
        self.poro_cnn = CNN100()
        self.satu_cnn = CNN20()
        self.pres_cnn = CNN20()
        self.batchnorm = torch.nn.BatchNorm1d(num_features=1447)
        
        self.lin1 = torch.nn.Linear(in_features=1447, out_features=500)
        self.lin2 = torch.nn.Linear(in_features=500, out_features=100)
        self.lin3 = torch.nn.Linear(in_features=100, out_features=30)
        self.ouput = torch.nn.Linear(in_features=30, out_features=1)
    
    def forward(self, Xperm, Xporo, Xsatu, Xpres, Xauxu):
        poro_out = self.poro_cnn(Xporo)
        perm_out = self.perm_cnn(Xperm)
        satu_out = self.satu_cnn(Xsatu)
        pres_out = self.pres_cnn(Xpres)
        #Concatenation
        con_out = torch.cat([perm_out,poro_out,satu_out,pres_out,Xauxu],1)
        con_out_norm = self.batchnorm(con_out)
        lin1_out = torch.nn.functional.relu(self.lin1(con_out_norm))
        lin2_out = torch.nn.functional.relu(self.lin2(lin1_out))
        lin3_out = torch.nn.functional.relu(self.lin3(lin2_out))
        fopt = self.ouput(lin3_out)  
        return fopt

In [19]:
### (2) Model
device= torch.device("cuda:0" if torch.cuda.is available() else "cpu")
net = Net()
net.load_state_dict(torch.load("my_model.pt"))
net.eva()
net = net.to(device)
cost_function = torch.nn.MSELoss() # Mean squared error

In [ ]:
### (3) Training / Fitting
prediction= net(pt_testing_X_perm,pt_testing_X_poro,pt_testing_X_satu,pt_testing_X_pres,pt_testing_X_auxu) # output of h(x)
loss= cost_function(prediction,pt_testing_Y) # average[(y-h(x))^2] L2 error
print(torch.mean(prediction-pt_testing_Y)*10000.0, "Testing MSE loss:", loss.data*10000)
    

In [ ]:
#L1 error:average[|y-h(x)|]

In [ ]:
torch.mean(torch.abs(pt_testing_Y-prediction))*10000

In [ ]:
# Mean difference average[y-h(x)]

In [ ]:
torch.mean(pt_testing_Y-prediction)*10000